In [1]:
from seaborn import heatmap
from matplotlib.pyplot import figure, savefig
from pandas import DataFrame
from numpy import ndarray
from requests import get
from time import sleep

%matplotlib inline

In [2]:
def get_markets_ndarray() -> ndarray:
    resp = get('https://api.pro.coinbase.com/products')
    if resp.status_code != 200:
        return None
    else:
        df = DataFrame(resp.json())
        return df[['id']].values

In [3]:
#markets = get_markets_ndarray()
#print (len(markets), markets)

In [4]:
def get_market_data(market: str='BTC-GBP', granularity: int=3600) -> DataFrame:
    resp = get(f'https://api.pro.coinbase.com/products/{market}/candles?granularity={str(granularity)}')
    if resp.status_code != 200:
        raise Exception(format(resp.json()['message']))
    else:
        df = DataFrame(resp.json(), columns=[ 'epoch', 'low', 'high', 'open', 'close', 'volume' ])
        df = df.iloc[::-1].reset_index()
        return df

In [5]:
#market_data = get_market_data('BTC-GBP', 3600)
#print (market_data)

In [6]:
def preprocess_market(market: str='BTC-GBP', granularity: int=3600) -> DataFrame:
    df = get_market_data(market, granularity)
    df['ma1'] = df.close.rolling(50, min_periods=1).mean()
    df['ma2'] = df.close.rolling(200, min_periods=1).mean()
    df['up'] = df['ma1'] > df['ma2']
    df['up'] = df['up'].astype(int)
    df = df[['up']]
    df = df.rename(columns={'up': market})
    return df.T

In [7]:
market_processed = preprocess_market('BTC-GBP', 3600)
print (market_processed)

         0    1    2    3    4    5    6    ...  293  294  295  296  297  298  299
BTC-GBP    0    0    0    0    0    0    0  ...    0    0    0    0    0    0    0

[1 rows x 300 columns]


In [ ]:
df = DataFrame()
for market in get_markets_ndarray():
    print ('Processing', market[0])
    
    tmp_df = df.append(preprocess_market(market[0], 3600))
    
    if len(tmp_df.T) == 300:
        df = df.append(preprocess_market(market[0], 3600))
        sleep(0.1) # avoid public rate exceeded

Processing 1INCH-EUR
Processing ANKR-GBP
Processing MKR-USD
Processing UNI-USD
Processing ETC-GBP
Processing SNX-GBP
Processing REN-BTC
Processing SOL-USDT
Processing NU-BTC
Processing ADA-EUR
Processing 1INCH-BTC
Processing NU-EUR
Processing COMP-USD
Processing OXT-USD
Processing OMG-EUR
Processing LINK-ETH
Processing DAI-USD
Processing BAND-BTC
Processing GRT-USD
Processing ICP-EUR
Processing GRT-BTC
Processing EOS-USD
Processing CGLD-BTC
Processing OMG-BTC
Processing CGLD-USD
Processing ICP-USD
Processing FORTH-USD
Processing LRC-BTC
Processing FIL-USD
Processing USDC-GBP
Processing XLM-EUR
Processing ADA-USDC
Processing LTC-GBP
Processing BTC-GBP
Processing USDT-USD
Processing OMG-GBP
Processing STORJ-USD
Processing COMP-BTC
Processing ZRX-BTC
Processing UMA-BTC
Processing ICP-GBP
Processing BAT-USDC
Processing UMA-GBP
Processing LINK-BTC
Processing ALGO-BTC
Processing XTZ-GBP
Processing WBTC-BTC
Processing BNT-GBP
Processing BCH-BTC
Processing NKN-BTC
Processing NKN-USD
Processing

In [ ]:
df.dropna(inplace=True)
df = df.astype(int)

In [ ]:
df

In [ ]:
df.to_csv('coinbasepro_market_analysis.csv')

In [ ]:
figure(figsize=(20,15))
heatmap(df.T.corr())

In [ ]:
figure(figsize=(30,50))
df_transpose = df.T
df_transpose.corr()['BTC-GBP'].sort_values(ascending=True).plot(kind='barh')
savefig('coinbasepro_market_analysis.jpg')

In [ ]:
top_10 = df_transpose.corr()['BTC-GBP'].sort_values(ascending=False).keys()[0:20]
print (top_10)

In [ ]:
bottom_10 = df_transpose.corr()['BTC-GBP'].sort_values(ascending=True).keys()[0:20]
print (bottom_10)

In [ ]:
df_filtered = df[~df_transpose.keys().str.contains('USD[TC]', regex=True)]
df_filtered_transpose = df_filtered.T

In [ ]:
bottom_filtered_10 = df_filtered_transpose.corr()['BTC-GBP'].sort_values(ascending=True).keys()[0:20]
print (bottom_filtered_10)